In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import *

import tensorflow as tf

from keras import backend as K

# Auxilary

# Model

## Building block

### Resnet

In [ ]:
def resnet_unit(feat_dim, kernel_size, x_in):
    # conv = Conv2D(feats, kernel, padding="same")
    res = keras.Sequential([
        Conv2D(feat_dim, kernel_size, padding="same"
            ,kernel_initializer='he_uniform'
            ,bias_initializer='he_uniform',
            kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
            ),
        ReLU(),
        Conv2D(feat_dim, kernel_size, padding="same", 
            kernel_initializer='he_uniform',
            bias_initializer='he_uniform',
            kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    ])
    return ReLU()(x_in + res(x_in))

def resnet_block(feat_dim, reps, x_in, pooling = True):
    # Stage 2
    conv1 = Conv2D(feat_dim, 3, padding="same", kernel_initializer='he_uniform',
            bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x_in)
    relu1 = ReLU()(conv1)
    conv2 = Conv2D(feat_dim, 3, padding="same", kernel_initializer='he_uniform',
            bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(relu1)
    x = ReLU()(conv2)
    for _ in range(reps):
        x = resnet_unit(feat_dim,3,x)
    if pooling == True:
        x = MaxPooling2D(2,2)(x)
        return x
    else:
        return x

### Convolution block

In [2]:
def conv_block(feat_dim, reps, x, kernel_size, mode = 'normal'):
    if mode == 'down':
        x = MaxPooling2D(2,2)(x)
    if mode == 'up':
        x = UpSampling2D((2,2))(x)
    # if mode == 'normal':
    for _ in range(reps):
        x = Conv2D(feat_dim, kernel_size, activation = LeakyReLU(0.2), padding="same", kernel_initializer='he_uniform', 
            bias_initializer='he_uniform')(x)
        x = Conv2D(feat_dim, 1, activation = LeakyReLU(0.2), padding="same", kernel_initializer='he_uniform',
            bias_initializer='he_uniform')(x)

    return x
    

### SPADE

In [ ]:
class SPADE(layers.Layer):
    def __init__(self, filters, epsilon=1e-5, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.conv = layers.Conv2D(filters, 3, padding="same", activation="relu",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))
        self.conv_gamma = layers.Conv2D(filters, 3, padding="same",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))
        self.conv_beta = layers.Conv2D(filters, 3, padding="same",kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))

    def build(self, input_shape):
        self.resize_shape = input_shape[1:3]

    def call(self, input_tensor, raw_mask):
        mask = tf.image.resize(raw_mask, self.resize_shape, method="nearest")
        x = self.conv(mask)
        gamma = self.conv_gamma(x)
        beta = self.conv_beta(x)
        mean, var = tf.nn.moments(input_tensor, axes=(0, 1, 2), keepdims=True)
        std = tf.sqrt(var + self.epsilon)
        normalized = (input_tensor - mean) / std
        output = gamma * normalized + beta
        return output

def spade_generator_unit(feats_in, feats_out, kernel, x, mask, upsampling = True):
    x = GaussianNoise(0.05)(x)
    # SPADE & conv
    spade1 = SPADE(feats_in)(x, mask)
    conv1 = Conv2D(feats_in,kernel, padding='same', activation= LeakyReLU(0.2),kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(spade1)
   
    conv_out = Conv2D(feats_out,kernel, padding='same', kernel_initializer='he_uniform',
               bias_initializer='he_uniform',kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(conv1)
    output = LeakyReLU(0.2)(conv_out)
    if upsampling == True:
        output = UpSampling2D(size = (2,2))(output)
        return output
    else:
        return output

### Adaptive Instance Normalization

In [ ]:
class AdaIN(layers.Layer):
    def __init__(self, filters, epsilon=1e-5, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.dense = layers.Dense(filters,
                                  activation = 'relu',
                                  kernel_initializer='he_uniform',
                                  bias_initializer='he_uniform')
        self.dense_gamma = layers.Dense(filters,
                                        kernel_initializer='he_uniform',
                                        bias_initializer='he_uniform')
        self.dense_beta = layers.Dense(filters,
                                       kernel_initializer='he_uniform',
                                       bias_initializer='he_uniform')

    def call(self, input_tensor, style_vector):
        x = self.dense(style_vector)
        gamma = self.dense_gamma(x)
        beta = self.dense_beta(x)
        #Normalize x[0]
        mean, var = tf.nn.moments(input_tensor, axes=(0, 1, 2), keepdims=True)
        std = tf.sqrt(var + self.epsilon)
        normalized = (input_tensor - mean) / std
        y = (x[0] - mean) / std

        #Reshape gamma and beta
        pool_shape = [-1, 1, 1, y.shape[-1]]
        gamma = tf.reshape(gamma, pool_shape) + 1.0
        beta = tf.reshape(beta, pool_shape)

        return gamma * normalized + beta

### Feature extraction CNN

In [3]:
def feature_extraction_unet(n_out_features = 128, n_base_features = 64, input_shape = (256,512,5)):
    inputs = keras.Input(shape = input_shape)
    conv1 = conv_block(feat_dim = n_base_features,
                       reps = 1, x = inputs,
                       kernel_size = 5)
    conv2 = conv_block(feat_dim = n_base_features*2,
                       reps = 1, x = conv1,
                       kernel_size = 5,
                       mode = 'down')
    conv3 = conv_block(feat_dim = n_base_features*4,
                       reps = 1, x = conv2,
                       kernel_size = 5,
                       mode = 'down')
    conv4 = conv_block(feat_dim = n_base_features*8,
                       reps = 1, x = conv3,
                       kernel_size = 5,
                       mode = 'down')
    conv5 = conv_block(feat_dim = n_base_features*16,
                       reps = 1, x = conv4,
                       kernel_size = 1,
                       mode = 'down')
    conv6 = conv_block(feat_dim = n_base_features*8,
                       reps = 1, x = conv5,
                       kernel_size = 1,
                       mode = 'up')
    # concat7 = Concatenate()([conv6,conv3])
    # conv7 = conv_block(feat_dim = n_base_features*4,
    #                    reps = 1, x = concat7,
    #                    kernel_size = 1,
    #                    mode = 'up')
    
    # conv8 = conv_block(feat_dim = n_base_features*2,
    #                    reps = 1, x = conv7,
    #                    kernel_size = 1,
    #                    mode = 'up')
    # concat9 = Concatenate()([conv8,conv1])
    # conv9 = conv_block(feat_dim = n_out_features,
    #                    reps = 1, x = concat9,
    #                    kernel_size = 1,
    #                    mode = 'normal')

    unet = keras.Model(inputs, conv6)
    return unet
feature_extraction_unet().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 512, 5)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 512, 64)      8064      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 512, 64)      4160      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 256, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 256, 128)     204928    
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 256, 128)     16512     
                                                             

In [ ]:


# Feature extraction CNN

# Differentiator CNN

# Integrator CNN

# PARCv2

# Data pipeline

# Training

# Validation